# How to load(export) data from(to) Google drive   

# Packages

In [0]:
from pandas import DataFrame as df
import numpy as np

!pip install -U -q PyDrive

# Data load

## 1. Load "Text" File

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
# 구글드라이브에 데이터가 들어있는 폴더주소:https://drive.google.com/drive/folders/폴더주소
# 폴더주소의 고유번호로 아래 코드를 바꿔주면 파일 리스트를 획득할 수 있음
file_list = drive.ListFile({'q': "'1xP6IN71nbA0lvqnK-GyovadsAEfV4ham' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: example.xlsx, id: 1UDcq-d5jdE8Vlv2dKtl6nZi8YD--ZOWG
title: example.txt, id: 1kB6uJShUiw48YlKfaCaMW4ZE4zvHXT1M


In [0]:
from google.colab import auth
from googleapiclient.discovery import build
import io , requests, os
import sys
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
# function for loading txt file
# ref: https://medium.com/@likho2manish/dev-steps-to-google-colab-5c72779c0ae9
def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    progress, done = downloader.next_chunk()
    if verbose:
      sys.stdout.flush()
      sys.stdout.write('\r')
      percentage_done = progress.resumable_progress * 100/progress.total_size
      sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded

In [10]:
# write down the file id you want to load from the file list you listed
down = get_file_buffer('1kB6uJShUiw48YlKfaCaMW4ZE4zvHXT1M')

# Load text content from the file
bookContent = down.getvalue().decode("utf-8")
print(bookContent)

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque at euismod urna, vel luctus lorem. Aliquam sem turpis, interdum blandit dui nec, consequat commodo eros. Donec hendrerit ex sed pellentesque mollis. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Integer orci ligula, placerat facilisis pretium quis, volutpat a diam. Suspendisse et nisi hendrerit nisi hendrerit interdum. Mauris mollis egestas nisi, et sagittis ligula ullamcorper vitae. Nam a convallis risus. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Quisque ullamcorper cursus tellus ac pulvinar. Curabitur eget libero ultricies, vehicula nunc ut, vehicula dui. Mauris at nulla non sapien eleifend imperdiet. Suspendisse eleifend a nisi sit amet porttitor.

Morbi dignissim sem nec sapien pellentesque, ac aliquet sem posuere. Nulla laoreet pretium odio in porta. Quisque pulvinar arcu scelerisque ipsum pretium, eget euismod augue vulputate.

In [35]:
# file to pandas dataframe
book_raw_array = bookContent.split("\n")
book_raw_array = df(book_raw_array)


# data preprocessing
# change data frame colum names
book_raw_array = book_raw_array.rename(index=str, columns={0: "contents"})

# remove empty cells
book_raw_array['contents'].replace('', np.nan, inplace=True)
book_raw_array = book_raw_array.dropna()

# rearrange index
book_raw_array = book_raw_array.reset_index()
book_raw_array = book_raw_array.drop('index', 1)

book_raw_array

,contents
0,"Lorem ipsum dolor sit amet, consectetur adipis..."
1,"Morbi dignissim sem nec sapien pellentesque, a..."
2,"Etiam quis erat dictum, imperdiet purus ut, so..."
3,Nunc condimentum condimentum mauris at maximus...
4,Aliquam quis aliquam enim. Praesent vestibulum...


## 2. Load "xlsx" file

In [18]:
# Write down the file name you want to use
file_id = '1UDcq-d5jdE8Vlv2dKtl6nZi8YD--ZOWG'
downloaded = drive.CreateFile({'id': file_id})
print('title: ', downloaded['title'], ', mimeType: ', downloaded['mimeType'])


# Download the file to a local disk as downloaded['title'].
mimetypes = {
    # Drive Document files as PDF
    'application/vnd.google-apps.document': 'application/pdf',
    
    # Drive Sheets files as MS Excel files.
    'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    
    # see https://developers.google.com/drive/v3/web/mime-types
}


# You must check the type before implmenting download
# Download should be done according to the MimeType you set.
# So, you should declare the file's mimetype.
download_mimetype = None
if downloaded['mimeType'] in mimetypes:
  print('the file has mimeType')
  download_mimetype = mimetypes[ downloaded['mimeType']]
  downloaded.GetContentFile(downloaded['title'], mimetype=download_mimetype)
  
else:
  print('the file does not have mimeType')
  downloaded.GetContentFile(downloaded['title'])
  
print("The file has been downloaded")

('title: ', u'example.xlsx', ', mimeType: ', u'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
the file does not have mimeType
The file has been downloaded


In [19]:
!pip install -q xlrd

import pandas as pd
book_raw_array_xlsx = pd.read_excel(downloaded['title'])
book_raw_array_xlsx

,Groot said
0,I am Groot!
1,I am Groot?
2,I am Groot.
3,I am Groot…
4,I am Grooooot
5,I am Great!


# Save data

## Export Data Frame to "xlsx(or csv)" file

In [0]:
# MimeTypes
# https://github.com/google/google-drive-proxy/blob/master/DriveProxy/API/MimeType.cs
# https://stackoverflow.com/questions/11894772/google-drive-mime-types-listing?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
mime_types= {
    "xls":'application/vnd.ms-excel',
    "xlsx":'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
    "xml":'text/xml',
    "ods":'application/vnd.oasis.opendocument.spreadsheet',
    "csv":'text/plain',
    "tmpl":'text/plain',
    "pdf": 'application/pdf',
    "php":'application/x-httpd-php',
    "jpg":'image/jpeg',
    "png":'image/png',
    "gif":'image/gif',
    "bmp":'image/bmp',
    "txt":'text/plain',
    "doc":'application/msword',
    "js":'text/js',
    "swf":'application/x-shockwave-flash',
    "mp3":'audio/mpeg',
    "zip":'application/zip',
    "rar":'application/rar',
    "tar":'application/tar',
    "arj":'application/arj',
    "cab":'application/cab',
    "html":'text/html',
    "htm":'text/html',
    "default":'application/octet-stream',
    "folder":'application/vnd.google-apps.folder'
}

In [0]:
# Export file.
# You should have Parent folder id if you want to organize the resources systematically
filename = "exported"
f = drive.CreateFile({'title': filename, 'mimeType': mime_types['xlsx'], "parents": [{"id": '1xP6IN71nbA0lvqnK-GyovadsAEfV4ham'}]})

f.SetContentString(book_raw_array.to_csv())

f.Upload({'convert': True})